### **Organize PSD Data**

- [ ]  Ensure all power spectral data (PSD) is saved per **subject-session** in a consistent format:
    - One file or object per subject-session
    - Contains:
        - EEG **channels**
        - **Frequencies**
        - Power per **epoch** or pre-averaged by condition
    - Associated **metadata**:
        - Subject ID
        - Session ID 
        - Dataset name
        - Cognitive **state label** (OT, MW)

### **Extract Alpha Power**

- [ ]  For each subject-session
    - Select **8–12 Hz** frequency band
    - **Sum power across frequencies** per channel and epoch
- [ ]  Result per epoch:

```python
{
    "subject_id": "001",
    "session_id": "1",
    "channel": "Pz",
    "state": "MW",
    "alpha_power": 3.45,
    ...
}
```

### **Assign Numerical Condition Labels**

For modeling the *ordinal direction* of alpha power:

| State | Code |
| --- | --- |
| OT | 0 |
| MW | 1 |
| MED | 2 |
- [ ]  For **Jin** and **Touryan**, you only use codes `0` and `1` (OT, MW)
- [ ]  For **Braboszcz**, use codes `1` and `2` (MW, MED)

### **Create Long-Form DataFrame per Dataset**

For each dataset, prepare:
| subject_session | subject_id | group | channel/ROI | state | alpha_power |
| --- | --- | --- | --- | --- | --- |
| 001_1 | 001 | NaN | Pz | 0 | 3.24 |
| 001_1 | 001 | NaN | Pz | 1 | 3.88 |
| 060_1 | 060 | vip | Pz | 1 | 5.12 |
| 060_1 | 060 | vip | Pz | 2 | 6.00 |

### **Fit Mixed Effects Model per Channel**

Do this **separately for each dataset** and for each channel (or ROI):

### ✅ Model formula:

```python
alpha_power ~ state + (state | subject_session)
```

This:

- Estimates the effect of `state` (e.g., OT → MW or MW → MED)
- Models per-subject variability in both baseline (intercept) and sensitivity (slope)

### **Store & Summarize Results**

For each model/channel:

- [ ]  Store:
    - Fixed effect estimate for `state`
    - p-value
    - t-statistic (optional)
    - Number of subjects included
- [ ]  Save results to `.csv` or `.json`

### **Visualize Results**

- [ ]  Plot per-channel **bar plots** or **line plots** of alpha power by condition
- [ ]  Plot **topoplots** (scalp maps) of:
    - `state` slope per channel
    - p-values (FDR-corrected or raw, with masking)
    - t-values

### **Correct for Multiple Comparisons**

- [ ]  Across channels (e.g., for scalp maps), correct p-values using:
    - **False Discovery Rate (FDR)**
    - Or **cluster-based permutation test** if possible

### **Filter Low-Effect Subjects**

- [ ]  Compute within-subject effect sizes (e.g., Cohen’s *d*) between states
- [ ]  Re-run the model excluding sessions with **d < 0.5**
- [ ]  Compare results to check robustness